In [1]:
%cd ..

/media/axeh/code_stuff/python_stuff/projects/reaction_diffusion_tutorial


In [2]:
import numpy as np
from scipy.ndimage import convolve
from tqdm.notebook import tqdm

from bokeh.plotting import figure
from bokeh.io import push_notebook, show, output_notebook
from bokeh.models import ColumnDataSource
from  ipywidgets import interact
output_notebook()

from recaps.utils import euler_forward, runge_kutta2, convert2img

Loading BokehJS ...

Gray-Scott system was already mentioned earlier in this notebook. Now we can finally code it! Extremely visually appealing reaction-diffusion tutorial which uses Gray-Scott system as an example is available [here](http://www.karlsims.com/rd.html) and super detailed Gray-Scott exploration guide is available at already mentioned [xmorphia](http://mrob.com/pub/comp/xmorphia/). Apart from that, there are [youtube](https://www.youtube.com/watch?v=oNHrWqvp8zc) [tutorials](https://www.youtube.com/watch?v=BV9ny785UNc) on Gray-Scott implementation in different software. With so many resources available I'll just brush over the main points behind the model quickly.

Gray-Scott system is yet another two-component system where the components $A$ and $B$ are engaged in a predator-prey type of relationship. Although in this system it takes __two__ predators $B$ to gang up against the prey $A$ and convert the latter into more $B$. Mathematically this translates into the net growth rate of $B$ being positively affected by $x_Ax_B^2$ term and net growth rate of $A$ being negatively affected by the same term. Additionally, prey $A$ is assumed to grow on some mysterious food source with the growth rate $f(1-x_A)$ (the $(1-x_A)$-part doesn't allow $A$ to grow beyond $1$), and predator $B$ is assumed to continuously die out with the decay rate $(k - f)x_B$. Of course, both species can also diffuse throught the environment. So all-in-all we have:

$$ \begin{cases}
   \frac{dx_A}{dt} = D_A\nabla^2x_A + f(1-x_A) - x_Ax_B^2 \\
   \frac{dx_B}{dt} = D_B\nabla^2x_B + x_Ax_B^2 - (k-f)x_B 
   \end{cases} $$
   
With this simple system, it is possible to generate a plethora of bizzarre patterns! The exact pattern would depend on a choice of parameters $f$ and $k$. Some examples of parameters and their corresponding patterns are listed in the code.

In [3]:
def grayscott(t, x, p):
    x = x.reshape(p['size'])   # resize flat 1D array back into (height,width,#channels)-array
    dxdt = np.zeros(p['size']) # preallocate the accumulation terms    
    diffusion = np.zeros(p['size']) # preallocate the diffusion term
    reaction  = np.zeros(p['size']) # preallocate the reaction term
    
    # --- get diffusion term ---
    # this time we'll take into account not only direct horizontal and vertical neighbours,
    # but the diagonal neighbours as well; 
    # they'll be assigned slightely lower weights, since they're a bit far after all
    kernel = np.array([[.05,  .20, .05], 
                       [.20, -1  , .20], 
                       [.05,  .20, .05]]) 
    for k in range(p['size'][2]):   
        diffusion[:,:,k] = p['D'][k]/p['h']**2 * convolve(x[:,:,k], kernel, mode="nearest")
        
    # --- get reaction term ---
    x_a, x_b = x[:,:,0], x[:,:,1]
    reaction[:,:,0] = -x_a*x_b**2 +  p['f']*(1 - x_a)
    reaction[:,:,1] =  x_a*x_b**2 - (p['k'] + p['f'])*x_b   
        
    # --- get total accumulation term ---
    dxdt = diffusion + reaction
        
    return dxdt.ravel()

# parameters: system 
# cells   : f = 0.0367, k = 0.0649
# corals  : f = 0.0545, k = 0.0620
# spirals : f = 0.0140, k = 0.0450
p = {'f' : 0.0367,
     'k' : 0.0649,
     'D' :[1.0, 0.5]}# diffusion rates 

# parameters: spatial grid
resolution = [100,100]        # grid dimensions in pixels
p['size']  = (*resolution, 2) # (height, width, #states)
p['h']     = 2                # pixel size in physical units


# time-related
t0, tf, dt = 0, 8000, 1
t_span = np.arange(t0, tf+dt, dt)

# initial condition
x_a0 = np.ones(resolution) 
x_b0 = np.zeros(resolution)
x_b0[np.random.randint(0,resolution[0],3), np.random.randint(0,resolution[1],3)] = 1 # random seeds (predator)
x0 = np.stack((x_a0, x_b0), axis=2).ravel()

# run the simulation: use euler_forward or runge_kutta2 
img_development = runge_kutta2(grayscott, x0, t_span, p);

  0%|          | 0/8000 [00:00<?, ?it/s]

In [4]:
# plot stuff!
img = np.zeros(p['size'][:2], dtype=np.uint32)
view = img.view(dtype=np.uint8).reshape((*p['size'][:2], 4))
view += np.flipud(convert2img(img_development[:,0], p['size']))

pb = figure(
    x_range=(0,p['size'][0]), 
    y_range=(0,p['size'][1]),
    plot_width=p['size'][0]*4,
    plot_height=p['size'][1]*4
)
r = pb.image_rgba(
    image=[img],
    x=0, 
    y=0, 
    dw=p['size'][0], 
    dh=p['size'][1]
)
handle = show(pb, notebook_handle=True)  

def update(t=0):
    i = int(t/dt)
    # update data_source
    #(adjust the contrst to make patterns more visible)
    img = np.zeros(p['size'][:2], dtype=np.uint32)
    view = img.view(dtype=np.uint8).reshape((*p['size'][:2], 4))
    view += np.flipud(convert2img(img_development[:,i], p['size'], steepness=10, midpoint=0.6))
    r.data_source.data['image'] = [img]
    push_notebook(handle=handle)
    
interact(update, t=(t0,tf,dt));

interactive(children=(IntSlider(value=0, description='t', max=8000), Output()), _dom_classes=('widget-interact…

Do try different combinations of parameters - the diversity of patterns that can be formed in this system is very rich! So far we haven't talked about the effect of parameters at all. That's mainly because it's a huge topic on its own and it definitely deserves [its own notebook](./../parameters2behaviour.ipynb).